In [61]:
input_data_path = '' #Input non-normalizad data
batch = ''
label = ''

Jupyter notebook for trvae integration algoritm

Author: Erno Hänninen

Title: run_trvae.ipynb

Created: 2023-01-08

In [62]:
import sys
#sys.path.insert(0, "../../Scripts") #Adding a path to be able to import the jupyter_functions
sys.path.insert(0, "../../../../Scripts") #Adding a path to be able to import the jupyter_functions
from jupyter_functions import *
import scib
import scanpy as sc
import torch
import scarches as sca
from scarches.dataset.trvae.data_handling import remove_sparsity
import matplotlib.pyplot as plt
import numpy as np
import gdown
import pandas




In [63]:
#Read the adata object and extract the variable genes to list
adata = sc.read(input_data_path)
hvgList = adata.var.index.tolist()
print(len(hvgList))

550


trVAE algorithm requires randomized batches of data and condition as input for training

In prediction the batches from source condition are transformed to target condtion

In [64]:
trvae_epochs = 50
surgery_epochs = 50

early_stopping_kwargs = {
    "early_stopping_metric": "val_unweighted_loss",
    "threshold": 0,
    "patience": 20,
    "reduce_lr": True,
    "lr_patience": 13,
    "lr_factor": 0.1,
}

In [65]:
#Split the dataset int reference and query dataset

#Process the data for trVAE
#adata = adata.raw.to_adata()
adata = remove_sparsity(adata) #if adata.X is sparse matrix -> converts it in to normal matrix

#Get source conditions (all batches of the data)
source_conditions = adata.obs[batch].unique().tolist()

In [ ]:
adata

In [ ]:
#Create the TRVAE model
trvae = sca.models.TRVAE(
    adata,
    condition_key=batch,
    conditions=source_conditions,
    hidden_layer_sizes=[128, 128],
)

In [ ]:
#Training trVAE with the reference dataset (source_adata)
trvae.train(
    n_epochs=trvae_epochs,  
    alpha_epoch_anneal=200,
    early_stopping_kwargs=early_stopping_kwargs,
    seed = 42
)

In [69]:
adata_latent = sc.AnnData(trvae.get_latent())
adata_latent.obs[label] = adata.obs[label].tolist()
adata_latent.obs[batch] = adata.obs[batch].tolist()

In [ ]:
sc.tl.pca(adata_latent)
sc.pp.neighbors(adata_latent, n_neighbors=8)
#sc.tl.leiden(adata_latent)
sc.tl.umap(adata_latent)
sc.pl.umap(adata_latent,
           color=[batch, label],
           frameon=False,
           wspace=0.6,
           )

In [ ]:
adata_latent

In [ ]:
adata_latent.write("../Integrated_adata/trvae_adata_final.h5ad")


In [ ]:
#Call the compute metrics function which computes the integration metrics
df = compute_metrics("trvae", adata, adata_latent, batch, label ,"X_pca", "full")
print(df)  